# AFSK Demodulator
## Step 2: Band-Pass FIR Filter

This is a Pynq portion of the AFSK demodulator project.  We will be using the FPGA overlay that we created in Vivado.

At this point we have created the bitstream for "project_02" and copied the bitstream, TCL wrapper, and hardware hand-off file to the Pynq board.

Let's first verify that we can load the module.

In [2]:
from pynq import Overlay, Xlnk
import numpy as np

overlay = Overlay('project_02.bit')
dma = overlay.bpfilter.bpf_dma

ModuleNotFoundError: No module named 'pynq'

## Accellerating FIR Filter

Below is the implementation of the AFSK demodulator in Python.  We are going to remove the band pass filter code and replace it with new code.

In [1]:
import sys
sys.path.append('../../base')

import numpy as np
from scipy.signal import lfiltic, lfilter, firwin
from scipy.io.wavfile import read
from DigitalPLL import DigitalPLL
from HDLC import HDLC
from AX25 import AX25
import time

from pynq import Overlay, Xlnk
import numpy as np

overlay = Overlay('project_02.bit')
dma = overlay.bpfilter.bpf_dma

block_size = 264

xlnk = Xlnk()
out_buffer = xlnk.cma_array(shape=(block_size,), dtype=np.int16)
in_buffer = xlnk.cma_array(shape=(block_size,), dtype=np.int16)

def bpf(data):
    start_time = time.time()
    output = np.array([],dtype=np.int16)
    for i in range(0, len(data), block_size):
        out_buffer[:len(data[i:i+block_size])] = data[i:i+block_size]
        dma.sendchannel.transfer(out_buffer)
        dma.recvchannel.transfer(in_buffer)
        dma.sendchannel.wait()
        dma.recvchannel.wait()
        output = np.append(output, in_buffer)
    stop_time = time.time()
    sw_exec_time = stop_time - start_time
    print('Hardware FIR execution time: ',sw_exec_time)
    return output

class fir_filter(object):
    def __init__(self, coeffs):
        self.coeffs = coeffs
        self.zl = lfiltic(self.coeffs, 32768.0, [], [])
    def __call__(self, data):
        result, self.zl = lfilter(self.coeffs, 32768.0, data, -1, self.zl)
        return result

class NRZI:

    def __init__(self):

        self.state = False

    def __call__(self, x):
        
        result = (x == self.state)
        self.state = x
        return result


audio_file = read('../../base/TNC_Test_Ver-1.102-26400-1sec.wav')
sample_rate = audio_file[0]
audio_data = audio_file[1]
delay = 12 # ~446us


lpf_coeffs = np.array(firwin(101, [760.0/(sample_rate/2)], width = None,
        pass_zero = True, scale = True, window='hann') * 32768, dtype=int)

lpf = fir_filter(lpf_coeffs)

filter_delay = 70

# Band-pass filter the audio data
f = np.append(bpf(audio_data[:26400]), bpf(np.zeros(filter_delay)))[filter_delay:]
# Digitize the data
d = np.array([int(x > 0) for x in f])
# Delay the data
a = d[delay:]
# XOR the digitized data with the delayed version
x = np.logical_xor(d[:0-delay], a)
# Low-pass filter the PWM signal
c = np.append(lpf(x-0.5), lpf(np.zeros(len(lpf_coeffs)//2)))[len(lpf_coeffs)//2:]
# Digitize the tone transistions
dx = np.array([int(x > 0) for x in c])
# Create the PLL
pll = DigitalPLL(sample_rate, 1200.0)

locked = np.zeros(len(dx), dtype=int)
sample = np.zeros(len(dx), dtype=int)

# Clock recovery
for i in range(len(dx)):
    sample[i] = pll(dx[i])
    locked[i] = pll.locked()
    
nrzi = NRZI()

data = [int(nrzi(x)) for x,y in zip(dx, sample) if y]

hdlc = HDLC()

for b,s,l in zip(dx, sample, locked):
    if s:
        packet = hdlc(nrzi(b), l)
        if packet is not None:
            print(AX25(packet[1]))


ModuleNotFoundError: No module named 'pynq'